In [0]:
data = [
    (1, "Alice", 1500, "2023-05-15"),
    (2, "Bob", 500, "2023-02-20"),
    (2, "Bob", 700, "2023-04-22"),
    (3, "Charlie", 1200, "2022-12-10"),
    (4, "Donald", 1200, "2024-12-10"),
    (4, "Donald", 1400, "2024-11-10"),
    (5, "Tom", 800, "2023-12-08")
]

schema = ['id', 'name', 'purchase_amount', 'purchase_date']

df = spark.createDataFrame(data, schema)


In [0]:
df.show()

In [0]:
from pyspark.sql.functions import *

df1 = df.filter(year(col('purchase_date')) == year(now())-1).filter(col('purchase_amount')>=1000).select('id', 'name').distinct()
df1.show()




In [0]:
data = [
    (111,'apple'),
    (111,'banana'),
    (123,'apple'),
    (123,'orange'),
    (124,'banana')
]

schema = ['id', 'name']

df=spark.createDataFrame(data,schema)

df.show()

In [0]:
from pyspark.sql.functions import collect_set, array_contains, size, col

result = (
    df.groupBy('id')
    .agg(
        collect_set('name').alias('fruits')
    )
    .filter(
        array_sort(col("fruits"))==array(lit("apple"),lit("banana"))
    )
    .select('id')
)

display(result)

In [0]:

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType
from datetime import date

spark = SparkSession.builder.appName("JobRunsExample").getOrCreate()

schema = StructType([
    StructField("job_status", StringType(), nullable=False),
    StructField("run_date", DateType(), nullable=False)
])

data = [
    ("success", date(2025, 8, 1)),
    ("success", date(2025, 8, 2)),
    ("fail", date(2025, 8, 3)),
    ("fail", date(2025, 8, 4)),
    ("success", date(2025, 8, 13))
]

df_job_runs = spark.createDataFrame(data, schema=schema)

In [0]:
from pyspark.sql.window import Window

job_status_window = Window.partitionBy(col('job_status')).orderBy('run_date')
window_all = Window.orderBy(col('run_date'))

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

job_status_window = Window.partitionBy(col('job_status')).orderBy('run_date')
window_all = Window.orderBy(col('run_date'))

df_final = (
    df_job_runs
    .withColumn('r1', row_number().over(job_status_window))
    .withColumn('r2', row_number().over(window_all))
)

display(df_final)